# Flood mapping

This notebook aims to step through the same process as Stefania's MATLAB code.

Initial steps are to load the relevant data (and ancilliaries).

In [1]:
import datacube, rasterio, numpy as np, matplotlib.pyplot as plt, xarray, pandas

In [4]:
sarcube = datacube.Datacube(config={'db_hostname': 'agdcdev-db.nci.org.au',
                                    'db_port': 6432,
                                    'db_database': 'radar_test'})

In [6]:
lat, lon = -27.1544, 149.0677 # surat
lat, lon = -28.034167, 148.581667 # st george

e = 0.01

In [5]:
radar = sarcube.load(product='s1_gamma0_scene', lat=(lat+e,lat-e), lon=(lon+e,lon-e))

,name,description,mission_id,gqa_abs_iterative_mean_xy,gqa_iterative_stddev_xy,lat,gqa,gqa_ref_source,time,product_type,...,relative_orbit,format,gqa_error_message,orbit,gqa_mean_xy,gqa_cep90,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,,,,,,,
52,dsm1sv10,DSM 1sec Version 1.0,NaN,NaN,NaN,NaN,NaN,NaN,None,DEM,...,NaN,GeoTIFF,NaN,NaN,NaN,NaN,EPSG:4326,"(-0.00027777777778, 0.00027777777778)",None,"(latitude, longitude)"
22,fc_percentile_albers_annual,"Landsat Fractional Cover percentile 25 metre, ...",NaN,NaN,NaN,NaN,NaN,NaN,None,fractional_cover_statistical_summary,...,NaN,NetCDF,NaN,NaN,NaN,NaN,EPSG:3577,"(-25, 25)","(100000.0, 100000.0)","(y, x)"
43,high_tide_comp_20p,High Tide 20 percentage composites for entire ...,NaN,NaN,NaN,NaN,NaN,NaN,None,tidal_composite,...,NaN,NetCDF,NaN,NaN,NaN,NaN,EPSG:3577,"(-25, 25)",None,"(y, x)"
44,high_tide_comp_count,High Tide 20 percentage pixel count,NaN,NaN,NaN,NaN,NaN,NaN,None,tidal_composite,...,NaN,NetCDF,NaN,NaN,NaN,NaN,EPSG:3577,"(-25, 25)",None,"(y, x)"
8,item_v2,Intertidal Extents Model,NaN,NaN,NaN,NaN,NaN,NaN,None,ITEM,...,NaN,NetCDF,NaN,NaN,NaN,NaN,EPSG:3577,"(-25, 25)",None,"(y, x)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,wofs_annual_summary,Water Observations from Space Annual Statistics,NaN,NaN,NaN,NaN,NaN,NaN,None,wofs_annual_summary,...,NaN,NetCDF,NaN,NaN,NaN,NaN,EPSG:3577,"(-25, 25)","(100000.0, 100000.0)","(y, x)"
10,wofs_apr_oct_summary,Water Observations from Space April to October...,NaN,NaN,NaN,NaN,NaN,NaN,None,wofs_apr_oct_summary,...,NaN,NetCDF,NaN,NaN,NaN,NaN,EPSG:3577,"(-25, 25)","(100000.0, 100000.0)","(y, x)"
49,wofs_filtered_summary,Water Observations from Space Statistics confi...,NaN,NaN,NaN,NaN,NaN,NaN,None,wofs_filtered_summary,...,NaN,NetCDF,NaN,NaN,NaN,NaN,EPSG:3577,"(-25, 25)","(100000.0, 100000.0)","(y, x)"
